# SaveModel Example – Using `ModelRecorder`
This notebook demonstrates how to use the `save_best_model` function to persist **PyTorch** and **scikit-learn** models to disk.

It covers:
- Training a small PyTorch model
- Training a scikit-learn model
- Saving both models using `ModelRecorder`
- Verifying the saved files

In [1]:
import os
import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from ThreeWToolkit.utils import ModelRecorder

In [2]:
from pathlib import Path

dir_to_save = Path("output/")
dir_to_save.mkdir(parents=True, exist_ok=True)

## Part 1 – Saving a PyTorch model

In [3]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(10, 2)

    def forward(self, x):
        return self.fc(x)


# Create random data
X = torch.randn(100, 10)
y = torch.randint(0, 2, (100,))

model = SimpleNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train for a few epochs
for epoch in range(5):
    outputs = model(X)
    loss = criterion(outputs, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

# Save the model
torch_model_path = dir_to_save / "torch_model.pt"
ModelRecorder.save_best_model(model, torch_model_path)

print(f"✅ PyTorch model saved to {torch_model_path}")

✅ PyTorch model saved to output/torch_model.pt


## Part 2 – Saving a scikit-learn model

In [4]:
X, y = make_classification(n_samples=100, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

sk_model = LogisticRegression()
sk_model.fit(X_train, y_train)

# Save the model
sk_model_path = dir_to_save / "sklearn_model.pkl"
ModelRecorder.save_best_model(sk_model, sk_model_path)

print(f"✅ scikit-learn model saved to {sk_model_path}")

✅ scikit-learn model saved to output/sklearn_model.pkl


## Part 3 – Verifying the saved files

In [5]:
assert os.path.exists(torch_model_path), "❌ Torch model file not found!"
assert os.path.exists(sk_model_path), "❌ scikit-learn model file not found!"

# Load back the scikit-learn model
with open(sk_model_path, "rb") as f:
    loaded_model = pickle.load(f)

print("✅ Loaded scikit-learn model type:", type(loaded_model))

✅ Loaded scikit-learn model type: <class 'sklearn.linear_model._logistic.LogisticRegression'>
